# Question to Copilot about how to do hyperparameter search in PyTorch

## Question 1: How to do dynamic hyperparameter test in PyTorch?

Answers below

### 1. Use Learning Rate Schedulers
PyTorch provides built-in schedulers in torch.optim.lr_scheduler to dynamically change the learning rate during training:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Example: Reduce LR on plateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)

for epoch in range(100):
    loss = torch.randn(1)  # Dummy loss
    optimizer.step()
    scheduler.step(loss)

### 2. Hyperparameter Sweeps with Libraries
For dynamic testing across multiple runs, use tools like:

Ray Tune
Optuna
Weights & Biases Sweeps

These allow you to:

Define a search space for hyperparameters.
Dynamically sample and adjust based on performance.

Example with Optuna:

In [ ]:
import optuna

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    # Train your PyTorch model here using lr and batch_size
    return validation_loss

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)


### 3. Dynamic Adjustment During Training
You can implement custom logic to adjust hyperparameters based on metrics:

In [ ]:
for epoch in range(num_epochs):
    train_loss = train_one_epoch(...)
    
    # Example: Adjust learning rate dynamically
    if train_loss < threshold:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.5

### 4. Combine with Callbacks
If you use PyTorch Lightning, you can add callbacks for dynamic hyperparameter tuning:

In [ ]:
from pytorch_lightning.callbacks import LearningRateMonitor

lr_monitor = LearningRateMonitor(logging_interval='step')
trainer = pl.Trainer(callbacks=[lr_monitor])

## Question 2: I already use: Example: Reduce LR on plateau scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10) should i use optune as well?

Answers below

Great question! If you’re already using ReduceLROnPlateau, that handles dynamic learning rate adjustment during training, which is good for convergence.
Optuna (or similar tools) serves a different purpose:

It optimizes hyperparameters across multiple training runs, not just the learning rate but also things like:

Initial learning rate
Batch size
Weight decay
Dropout rate
Model architecture parameters (e.g., number of layers, hidden units)



So:

ReduceLROnPlateau = dynamic adjustment within a single run
Optuna = systematic search across runs

If your goal is finding the best overall hyperparameter set, Optuna is worth adding. If you’re only fine-tuning LR during training, then your current setup is enough.
✅ Best practice: Use Optuna to find a good starting point for hyperparameters, then keep ReduceLROnPlateau for fine-grained control during training.

Do you want me to:

Show you a full Optuna integration example with PyTorch, including ReduceLROnPlateau inside the trial?
Or give you a comparison of Optuna vs Ray Tune vs manual grid search for your case?

## Question 3: Full Optuna integration example with PyTorch

Answers below

Here’s a full Optuna integration example with PyTorch, including ReduceLROnPlateau inside the trial loop:

✅ Complete Example: Optuna + PyTorch + ReduceLROnPlateau

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

# Dummy dataset
X = torch.randn(1000, 10)
y = torch.randn(1000, 1)
dataset = TensorDataset(X, y)

# Simple model
class SimpleModel(nn.Module):
    def __init__(self, hidden_size, dropout):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(10, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.fc2(x)

# Objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    hidden_size = trial.suggest_int('hidden_size', 32, 256)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)

    # DataLoader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Model, optimizer, scheduler
    model = SimpleModel(hidden_size, dropout)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    criterion = nn.MSELoss()

    # Training loop
    model.train()
    for epoch in range(20):
        epoch_loss = 0
        for batch_X, batch_y in loader:
            optimizer.zero_grad()
            preds = model(batch_X)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(loader)
        scheduler.step(avg_loss)

    # Validation metric (here using training loss as proxy)
    return avg_loss

# Run Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Best hyperparameters:", study.best_params)

What this does

Optuna searches over:

hidden_size (model architecture)
dropout
lr (initial learning rate)
batch_size
weight_decay


Inside each trial:

We train for 20 epochs.
Use ReduceLROnPlateau to dynamically adjust LR during training.


After trials, Optuna gives the best hyperparameter set.


Next Steps

Replace the dummy dataset with your real dataset.
Add a proper validation split and use validation loss for scheduler.step() and Optuna’s return value.
Optionally integrate early stopping and pruning with Optuna for faster optimization:

In [ ]:
trial.report(avg_loss, epoch)
if trial.should_prune():
    raise optuna.TrialPruned()

Do you want me to extend this example with Optuna pruning and PyTorch Lightning integration for a more production-ready setup? Or keep it vanilla PyTorch but add validation and early stopping?

## Question 4: I do train as

Answers below


Parameters for - train model
learning_rate = 0.001
optimizer = torch.optim.Adam(model_flexible.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
epochs = 30

Train with early stopping + scheduler
history = train_model(
    model=model_flexible,
    num_epochs=epochs,
    train_dl=train_dataloader,
    valid_dl=validation_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device='cpu',
    verbose=True,
    patience=5
)

and set model with
model_flexible = FlexibleCNN(
    input_size=input_size_img,
    num_classes=num_classes,
    conv_layers=[(32, 3), (64, 3), (128, 3)],
    fc_layers=[256, 128],
    activation=nn.ReLU,    # Custom activation
    dropout_fc=0.3,             # Dropout in FC layers
    dropout_conv=0.1,           # Dropout in conv layers
    use_batchnorm=True,
    pool_type="avg",            # Use AvgPool instead of MaxPool
    global_pool="max"           # Use AdaptiveMaxPool for final pooling
)


where 


class FlexibleCNN(nn.Module):
    def __init__(self, input_size, num_classes,
                 conv_layers=[(32, 3), (64, 3), (128, 3)],  # [(out_channels, kernel_size), ...]
                 fc_layers=[256, 128],                     # [units for each FC layer]
                 activation=nn.ReLU,                       # Activation class
                 dropout_fc=0.5,                           # Dropout for FC layers
                 dropout_conv=0.0,                         #  Optional dropout for conv layers
                 use_batchnorm=True,
                 pool_type="max",                          # "max" or "avg"
                 global_pool="avg",                        #  "avg" or "max" for final pooling
                 show_summary=True):
        super().__init__()

        # Choose pooling layers
        pool_layer = nn.MaxPool2d(2) if pool_type == "max" else nn.AvgPool2d(2)
        global_pool_layer = nn.AdaptiveAvgPool2d((1, 1)) if global_pool == "avg" else nn.AdaptiveMaxPool2d((1, 1))

        # Build convolutional feature extractor
        layers = []
        in_channels = input_size[0]
        for out_channels, kernel_size in conv_layers:
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2))
            if use_batchnorm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(activation())
            if dropout_conv > 0:
                layers.append(nn.Dropout2d(dropout_conv))  # ✅ Dropout for conv layers
            layers.append(pool_layer)
            in_channels = out_channels

        layers.append(global_pool_layer)  # ✅ Global pooling
        self.features = nn.Sequential(*layers)

        # Classifier
        in_features = conv_layers[-1][0]
        fc = []
        for units in fc_layers:
            fc.append(nn.Linear(in_features, units))
            fc.append(activation())
            if dropout_fc > 0:
                fc.append(nn.Dropout(dropout_fc))
            in_features = units
        fc.append(nn.Linear(in_features, num_classes))
        self.classifier = nn.Sequential(*fc)

        # Optional summary
        if show_summary:
            summary(self, input_size=(1, *input_size), col_names=["input_size", "output_size", "num_params"])

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

Thanks for sharing your setup! You already have a custom CNN architecture, early stopping, and a scheduler in your train_model function. To integrate Optuna with this workflow, we’ll wrap your training logic inside an Optuna objective function and make the hyperparameters dynamic.
Here’s how you can do it:

✅ Optuna Integration with Your FlexibleCNN and train_model


In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim

# Objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    dropout_fc = trial.suggest_float('dropout_fc', 0.1, 0.5)
    dropout_conv = trial.suggest_float('dropout_conv', 0.0, 0.3)
    activation_choice = trial.suggest_categorical('activation', [nn.ReLU, nn.LeakyReLU])
    batch_size = trial.suggest_int('batch_size', 16, 128)
    conv_layers = [(trial.suggest_int('conv1', 16, 64), 3),
                   (trial.suggest_int('conv2', 32, 128), 3),
                   (trial.suggest_int('conv3', 64, 256), 3)]
    fc_layers = [trial.suggest_int('fc1', 128, 512),
                 trial.suggest_int('fc2', 64, 256)]

    # Create model with dynamic architecture
    model_flexible = FlexibleCNN(
        input_size=input_size_img,
        num_classes=num_classes,
        conv_layers=conv_layers,
        fc_layers=fc_layers,
        activation=activation_choice,
        dropout_fc=dropout_fc,
        dropout_conv=dropout_conv,
        use_batchnorm=True,
        pool_type="avg",
        global_pool="max",
        show_summary=False
    )

    # Optimizer and loss
    optimizer = optim.Adam(model_flexible.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()

    # DataLoader with dynamic batch size
    train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)

    # Train with early stopping and scheduler
    history = train_model(
        model=model_flexible,
        num_epochs=30,
        train_dl=train_dl,
        valid_dl=valid_dl,
        loss_fn=loss_fn,
        optimizer=optimizer,
        device='cpu',
        verbose=False,
        patience=5
    )

    # Use best validation loss as metric
    best_val_loss = min(history['val_loss'])
    trial.report(best_val_loss, step=0)

    # Enable pruning
    if trial.should_prune():
        raise optuna.TrialPruned()

    return best_val_loss

# Run Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Best hyperparameters:", study.best_params)

Key Features

Dynamic architecture tuning: Conv layer sizes, FC layer sizes, dropout, activation.
Dynamic training hyperparameters: Learning rate, batch size.
Early stopping + scheduler: Still used inside train_model.
Optuna pruning: Stops bad trials early for efficiency.


✅ Next Steps

Replace train_dataset and valid_dataset with your actual datasets.
Add GPU support (device='cuda' if available).
Log results with Optuna dashboard (optuna-dashboard).


👉 Do you want me to add Optuna’s pruning callbacks for PyTorch Lightning, or keep it vanilla PyTorch but include visualization of the search results (e.g., hyperparameter importance, optimization history)?